In [2]:
import petl as etl
import pandas as pd

from utils.paths import DATA, SITE

In [3]:
TICKET_DATA = SITE / 'themes/ticketing/_data/operational'
TICKET_DATA.mkdir(parents=True, exist_ok=True)

In [4]:
tickets = etl.fromcsv(DATA / 'ticketing/tickets.csv').convertnumbers()

In [21]:
def make_date_index(df):
    df.date=df.date.pipe(pd.DatetimeIndex)
    return df

In [22]:
def resample_to_week(df):
    df = df.resample('W-FRI', on='date').sum()
    df.index.names = ['week_ending']
    return df

In [23]:
by_week = (
    tickets
    .selecteq('geography_type', 'oslaua')
    .aggregate('date', sum, 'count_of_tickets')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .todataframe()
    .pipe(make_date_index)
    .pipe(resample_to_week)
)

In [24]:
weekly_tickets = pd.concat(
    [
        by_week.rename(columns={'value': 'weekly'}),
        by_week.cumsum().rename(columns={'value': 'total'})
    ],
    axis=1
)

In [8]:
weekly_tickets.to_csv(TICKET_DATA / 'weekly_tickets.csv')

In [9]:
geo = tickets.aggregate(['geography_type', 'geography_code'], sum, 'count_of_tickets')

In [10]:
geo.tocsv(TICKET_DATA / 'geography.csv')

In [28]:
orders = (
    etl
    .fromcsv(DATA / 'ticketing/orders.csv')
    .convertnumbers()
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .todataframe()
    .pipe(make_date_index)
    .pipe(resample_to_week)
)


In [32]:
weekly_orders = pd.concat(
    [
        orders.rename(columns={'count_of_orders': 'weekly'}),
        orders.cumsum().rename(columns={'count_of_orders': 'total'})
    ],
    axis=1
)

In [33]:
weekly_orders.to_csv(TICKET_DATA / 'weekly_orders.csv')